In [ ]:
### Binary classification -> Neural networks.
###classifying circles

In [ ]:
##dependencies
from sklearn.datasets import make_circles #generate circles
import pandas as pd ##creating data frame
import matplotlib.pyplot as plt ##for drawing plots
import torch #creating tensors
from torch import nn ## basic building block of neural networks

In [ ]:
## make   1000 samples 
n_samples = 1000
x, y = make_circles(
    n_samples,
    noise=0.02,
    random_state=30
)

In [ ]:
## visualizing my data
##dataframe
circles = pd.DataFrame({
    'X1': x[:, 0], 
    'X2': x[:, 1],
    'Labels': y
})

circles.head(10)
##circles['Labels'].value_counts()

In [ ]:
### drawing plots
plt.scatter(
    x=x[:,0],
    y=x[:,1],
    s=8,
    c=y,
    cmap=plt.cm.RdYlBu
)
plt.show()

In [ ]:
##turn data into training and testing data
###input and output shapes 
##x.shape, y.shape

x_tensor = torch.from_numpy(x).type(torch.float)
y_tensor = torch.from_numpy(y).type(torch.float)

#x_tensor.shape, y_tensor.shape
#x_tensor, y_tensor

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    x_tensor,
    y_tensor, 
    test_size=0.2, #20% test data, 80% training data
    random_state=30
)

print(f"X training data: {len(X_train)}\nX Test Data: {len(X_test)}\n Y_train Data: {len(Y_train)}\n Y Test DATA: {len(Y_test)}")

In [ ]:
##device agnostic
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
### create a model
model_0 = nn.Sequential(
    nn.Linear(in_features=2, out_features=5), #2 inputs (x) with 1 hidden layer (5 neurons)
    nn.Linear(in_features=5, out_features=1) #1 output y
).to(device)

model_0

In [ ]:
# Make predictions with the model
untrained_preds = model_0(X_test.to(device))
print(f"Length of predictions: {len(untrained_preds)}, Shape: {untrained_preds.shape}")
print(f"Length of test samples: {len(Y_test)}, Shape: {Y_test.shape}")
print(f"\nFirst 10 predictions:\n{untrained_preds[:10]}")
print(f"\nFirst 10 test labels:\n{Y_test[:10]}")

In [ ]:
##loss function and optimizer
#loss_fn = torch.nn.BCELoss() # no sigmoid built in
loss_fn = torch.nn.BCEWithLogitsLoss() #sigmoid built in

#optimizer
optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.1)

#function to calculate accuracy of model
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() #find number of correct predictions
    acc = (correct/len(y_pred))*100
    return acc

In [ ]:
##logits are unnormalized final scores of your model -> source: data science stack exchange
##training and testing our model

torch.manual_seed(40)

epochs = 300 #train with 100 epochs 

#put train data and test data to device
X_train, Y_train = X_train.to(device), Y_train.to(device)
X_test, Y_test = X_test.to(device), Y_test.to(device)

#training loop

for epoch in range(epochs):
    model_0.train() #training mode

    #forward pass
    y_logits = model_0(X_train).squeeze() #squeeze to remove extra dimensions
    y_pred = torch.round(torch.sigmoid(y_logits)) #normalize using sigmoid fxn -> round to get prediction probabalities using round

    #calculate the loss
    loss = loss_fn(y_logits, Y_train) #not using sigmoid since BCEWithLogitsLoss has inbuilt sigmoid 

    acc = accuracy_fn(y_true=Y_train, 
    y_pred=y_pred)

    #optimizer zero grad
    optimizer.zero_grad()

    #loss backwards 
    loss.backward()

    #optmizer step -> apply gradient descent
    optimizer.step()

    ###testing 
    model_0.eval()
    with torch.inference_mode():
        #inference
        test_logits = model_0(X_test).squeeze()
        test_pred = torch.round(torch.sigmoid(test_logits))
        
        #calculate test loss
        test_loss = loss_fn(test_logits, Y_test) 

        #test accuracy
        test_acc = accuracy_fn(
            y_true=Y_test,
            y_pred=test_pred
        )

    #print out whats happening 
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} ~ Loss: {loss:.5f}, Accuracy: {acc:.2f} ~ Test Loss: {test_loss:.5f}, Test Accuracy: {test_acc:.2f}")

In [ ]:
#visualizing predictions
import requests
from pathlib import Path 
'''
# Download helper functions from Learn PyTorch repo (if not already downloaded)
if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download")
else:
  print("Downloading helper_functions.py")
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)
'''

from helper_functions import plot_predictions, plot_decision_boundary